# Organize directories

The first step in training a ML model on sequences of images acquired from the ProtectCode app is to ensure the directory structure is set up for subsequent data loading. 

These commands assume the following: 

1) That the top directory for the training data and this notebook are both located in ~/src/YPB-AI/ on the EC2 instance. To get a zipped directory from S3 to that directory on your EC2 instance, you can run the following bash command: "aws s3 cp s3://your-bucket-name/yourZipFile.zip ~/src/YPB-AI/yourZipFile.zip". And to unzip it, you can run "unzip yourZipFile.zip"

2) That the jupyter notebook server was opened from the ~/src/ directory on the EC2 instance via the command "jupyter notebook --ip=0.0.0.0 --no-browser".

3) That underneath the top directory (which I usually call "Training" or something of the like), there are a series of subdirectories named and organized by classification label. For instance, each subdirectory that contains true negative image sequences, according to my implementation, contains the word "Negative". And subdirectories with true positives do not contain the word "Negative". 



In [2]:
import numpy as np
from glob import glob
import os, sys
import re

Set the name of your top data directory below. 

In [9]:
# topDirName = 'Training'
# topDirName = 'Training2'
topDirName = '/home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20'

Lump together all subdirectories with the same classification label. 

In [13]:
#Assumes current working directory is ~/src/YPB-AI
allSamples = np.array(glob(topDirName + '/*'))
os.system('mkdir ' + topDirName + '/Absent')
os.system('mkdir ' + topDirName + '/Present')

for sample in allSamples: 
    if 'Negative' in sample or 'Sat' in sample:
        cmdString = "mv " + sample + " " + topDirName + "/Absent"
    else:
        cmdString = "mv " + sample + " " + topDirName + "/Present"
    os.system(cmdString)

Each directory within the classification subdirectory contains a series of folders that have the following contents:

1) A .txt file containing some timing and other metadata about the app run.

2) A folder called 'Photos' which contains the sequence of images acquired.

The next cell will obliterate the middle directory (the directory just below the classification directory) and repackage each run's photos into a uniquely identifiable folder of its own (lacking the text file) at the level where the middle directories once sat.



In [15]:
allPresentSamples = np.array(glob(topDirName + '/Present/*/*'))
for sample in allPresentSamples:
    digestedSample = re.split('/',sample)
digestedSample

['',
 'home',
 'ubuntu',
 'src',
 'YPB-AI',
 'beforeSplit',
 'CIFAR',
 '17_01_20',
 'Present',
 'XS_filledTube',
 'ProtectCodeScannerResults_27-Nov-2019_13-38-04']

In [16]:
### Find 'vestigial' directories to be deleted after unpacking.
presentVestDirs = np.array(glob(topDirName + '/Present/*'))
absentVestDirs = np.array(glob(topDirName + '/Absent/*'))

# Unpack the JPG photo files into identifiable directories.
allPresentSamples = np.array(glob(topDirName + '/Present/*/*'))
for sample in allPresentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    # digestedTimestamp = re.split('_',digestedSample[3])
    digestedTimestamp = re.split('_',digestedSample[10])
    # newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    newDir = digestedSample[9] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Present/' + newDir
    os.system(cmdString)
    
allAbsentSamples = np.array(glob(topDirName + '/Absent/*/*'))
for sample in allAbsentSamples:
    # Grab the unique identifiers -- phone model, sample # and date/time
    digestedSample = re.split('/',sample)
    # digestedTimestamp = re.split('_',digestedSample[3])
    digestedTimestamp = re.split('_',digestedSample[10])
    # newDir = digestedSample[2] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    newDir = digestedSample[9] + '_' + digestedTimestamp[1] + '_' + digestedTimestamp[2]
    # Make the new directory
    cmdString = 'mkdir ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    # Add the photos directly to it
    cmdString = 'mv ' + sample + '/Photos/*' + ' ' + topDirName + '/Absent/' + newDir
    os.system(cmdString)
    
    
# Delete vestigial directories.
for direc in presentVestDirs:
    os.system('rm -rf ' + direc)
    
for direc in absentVestDirs:
    os.system('rm -rf ' + direc)

Look through all directories and find image sequences containing the file "IMG_9999.JPG". If found, sort and rename to an appropriate image sequence. This is necessary because when iOS reaches "IMG_9999.JPG" it wraps back around to "IMG_0000.JPG" for the next in the sequence. If this is left as is, the sorting operation that follows will sort the images into the incorrect timing order. 

In [17]:
# Get the filenames.
imgSeqsToCorrect = os.popen('cd ' + topDirName + '; find -name "*IMG_9999.JPG"').read()
# Split into unique entities.
imgSeqsToCorrect = re.split('\n',imgSeqsToCorrect)
# Drop the last entry, it's empty.
imgSeqsToCorrect.pop();

# Change any files that don't start with 'IMG_9' (i.e., any files that come after the wraparound) to start with 'J'.
# This will mean that when they are sorted, they will come correctly after the wraparound.
for sample in imgSeqsToCorrect:
    digestedSample = re.split('/',sample)
    # pathToDir = os.path.join(digestedSample[0],digestedSample[1],digestedSample[2],digestedSample[3])
    pathToDir = os.path.join(topDirName,digestedSample[1],digestedSample[2])
    # pathToDir = topDirName + '/' + digestedSample[0] + '/' + digestedSample[1] + '/' + digestedSample[2] + '/' + digestedSample[3]
    try:
        filesInDir = os.listdir(pathToDir)
        for i in range(len(filesInDir)):
            if 'IMG_9' not in filesInDir[i]:
                newFile = filesInDir[i].replace('I','J',1)
                # cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
                cmdString = 'mv ' + os.path.join(pathToDir,filesInDir[i]) + ' ' + os.path.join(pathToDir,newFile)
                # cmdString = 'mv ' + '/' + filesInDir[i] + '/' + ' ' + pathToDir + '/' + newFile
                os.system(cmdString)
    except Exception as e:
        print(e)

# Change the images in the folder into a stack of 12-image input

In [18]:
import torch
import torchvision.models as models
from PIL import Image
import pickle

In [19]:
topdir = topDirName
ndx = 0
topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']

# Calculate the total number of stacks to transform.
numStacks = 0
for classDir in topdirs:
    thisClassDir = os.path.join(topdir,classDir)
    directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
    numStacks += len(directories)

for classDir in topdirs:
    thisClassDir = os.path.join(topdir,classDir)
    directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']


    # Do the transformation.
    for photoDir in directories:

        thisPhotoDir = os.path.join(thisClassDir, photoDir)
        ##pin pin fix##

#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue

        if not os.path.isdir(thisPhotoDir):
            continue

        ## end ##
        files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
        print("Transforming image stack {}".format(thisPhotoDir))
        files.sort()
        output = []
        # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
        for file in files:
            thisFile = os.path.join(thisPhotoDir,file)
            imgArr = np.float32(np.array(Image.open(thisFile).resize((32,32))).reshape([1,3,32,32]))
            imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
            imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
            imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
            output.append(imgArr)

        FinalOutput = torch.FloatTensor(output)
        # for i in range(len(files)):
        #    FinalOutput[i] = output[i]
        pathStr = thisPhotoDir + '.pickle'
        with open(pathStr,'wb') as f:
            pickle.dump(FinalOutput,f)
            ndx += 1
            print("{}% complete.".format(ndx/numStacks*100))
    

Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/XS_filledTube_Negative_27-Nov-2019_23-48-41
0.007735148514851485% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_Nikeshoes_Whole_Negative_30-Nov-2019_18-04-25
0.01547029702970297% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_Nikeshoes_Whole_Negative_30-Nov-2019_16-35-20
0.023205445544554455% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/7_NikeShoes_Negative_05-Dec-2019_03-38-15
0.03094059405940594% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_filledGreenTube_Negative_29-Nov-2019_15-31-21
0.038675742574257425% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_NikeShoes_Orange_Negative_26-Nov-2019_16-27-32
0.04641089108910891% complete.
Transforming image stack /home

0.4099628712871287% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/XS_filledTube_Negative_27-Nov-2019_21-45-23
0.41769801980198024% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/7_NikeShoes_Negative_05-Dec-2019_00-31-22
0.4254331683168317% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_filledGreenTube_Negative_29-Nov-2019_15-56-18
0.4331683168316832% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_greenBall_Negative_04-Dec-2019_11-11-45
0.44090346534653463% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/XS_filledTube_Negative_27-Nov-2019_23-34-31
0.4486386138613861% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_filledGreenTube_Negative_29-Nov-2019_14-25-48
0.4563737623762376% complete.
Transforming image 

0.812190594059406% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/7_NikeShoes_Negative_06-Dec-2019_11-44-22
0.8199257425742574% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_redTube_Negative_26-Nov-2019_11-37-05
0.827660891089109% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/7_NikeShoes_Negative_04-Dec-2019_22-34-07
0.8353960396039605% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_filledGreenTube_Negative_29-Nov-2019_18-44-33
0.8431311881188119% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/7_NikeShoes_Negative_04-Dec-2019_22-05-19
0.8508663366336634% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/XS_filledTube_Negative_27-Nov-2019_22-46-38
0.8586014851485149% complete.
Transforming image stack /home/ubu

1.2221534653465347% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/7_NikeShoes_Negative_05-Dec-2019_05-06-49
1.2298886138613863% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_filledGreenTube_Negative_29-Nov-2019_15-51-30
1.2376237623762376% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_NikeShoes_Orange_Negative_26-Nov-2019_20-59-13
1.2453589108910892% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_filledGreenTube_Negative_29-Nov-2019_12-12-22
1.2530940594059408% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/XS_filledTube_Negative_27-Nov-2019_21-54-42
1.2608292079207921% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_filledGreenTube_Negative_25-Nov-2019_16-48-02
1.2685643564356437% complete.
Transform

1.6321163366336635% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/7_NikeShoes_Negative_06-Dec-2019_13-31-31
1.6398514851485149% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/7_NikeShoes_Negative_05-Dec-2019_06-30-32
1.6475866336633664% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_filledGreenTube_Negative_25-Nov-2019_15-01-49
1.655321782178218% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/7_greenBall_Negative_03-Dec-2019_22-12-15
1.6630569306930694% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_NikeShoes_Orange_Negative_26-Nov-2019_23-27-07
1.670792079207921% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Absent/6S_filledGreenTube_Negative_29-Nov-2019_12-15-59
1.6785272277227723% complete.
Transforming image s

...

Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_13-Dec-2019_20-37-17
98.05074257425743% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/XS_filledTube_27-Nov-2019_13-09-17
98.05847772277228% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/6S_filledTube_28-Nov-2019_13-08-06
98.06621287128714% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_11-Dec-2019_21-00-43
98.07394801980197% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_13-Dec-2019_12-14-08
98.08168316831683% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/6S_filledTube_28-Nov-2019_14-22-03
98.08941831683168% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_13-Dec-2019_19-21-26
98.09

98.50711633663366% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_09-Dec-2019_14-12-05
98.51485148514851% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/6S_filledTube_28-Nov-2019_13-45-51
98.52258663366337% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_09-Dec-2019_22-15-12
98.53032178217822% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_11-Dec-2019_19-56-08
98.53805693069307% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/6S_filledTube_28-Nov-2019_17-45-44
98.54579207920791% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/XS_filledTube_27-Nov-2019_12-50-28
98.55352722772277% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/XS_fille

98.97122524752476% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/XS_filledTube_27-Nov-2019_14-58-43
98.97896039603961% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_09-Dec-2019_17-43-06
98.98669554455446% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_11-Dec-2019_23-50-53
98.9944306930693% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_NikeShoes_22-Nov-2019_12-57-39
99.00216584158416% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_11-Dec-2019_23-03-29
99.00990099009901% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/6S_filledTube_28-Nov-2019_17-43-52
99.01763613861387% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_

99.43533415841584% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_09-Dec-2019_14-59-53
99.4430693069307% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/XS_filledTube_27-Nov-2019_13-00-00
99.45080445544554% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/XS_filledTube_27-Nov-2019_16-23-52
99.45853960396039% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/XS_filledTube_27-Nov-2019_15-20-49
99.46627475247524% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/6S_filledTube_28-Nov-2019_16-20-17
99.4740099009901% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_NikeShoes_22-Nov-2019_10-42-36
99.48174504950495% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/XS_fil

99.86850247524752% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/6S_filledTube_28-Nov-2019_14-13-30
99.87623762376238% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_NikeShoes_22-Nov-2019_11-10-27
99.88397277227723% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_12-Dec-2019_20-11-21
99.89170792079209% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_13-Dec-2019_17-42-29
99.89944306930693% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_09-Dec-2019_23-05-17
99.90717821782178% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/7_whiteCap_09-Dec-2019_18-29-05
99.91491336633663% complete.
Transforming image stack /home/ubuntu/src/YPB-AI/beforeSplit/CIFAR/17_01_20/Present/6S_filledTube

Define the function that will transform model output into a pickle file.

In [20]:
def transform_to_pickle(model,topdir):
    model.eval()
    ndx = 0
    topdirs = [direc for direc in os.listdir(topdir) if direc[0] is not '.']
    
    # Calculate the total number of stacks to transform.
    numStacks = 0
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        numStacks += len(directories)
    
    for classDir in topdirs:
        thisClassDir = os.path.join(topdir,classDir)
        directories = [direc for direc in os.listdir(thisClassDir) if direc[0] is not '.']
        
        
        # Do the transformation.
        for photoDir in directories:
            
            thisPhotoDir = os.path.join(thisClassDir, photoDir)
            ##pin pin fix##
            
#             print(thisPhotoDir)
#             print(os.path.isfile(photoDir))
#             continue
            
            if not os.path.isdir(thisPhotoDir):
                continue

            ## end ##
            files = [photofile for photofile in os.listdir(thisPhotoDir) if photofile.lower().endswith('jpg')]
            print("Transforming image stack {}".format(thisPhotoDir))
            files.sort()
            output = []
            # Make sure to appropriately transform the image size and normalize the RGB values, as the model expects. 
            for file in files:
                thisFile = os.path.join(thisPhotoDir,file)
                imgArr = np.float32(np.array(Image.open(thisFile).resize((224,224))).reshape([1,3,224,224]))
                imgArr[0][0] = ((imgArr[0][0] - 123.675)/58.395)
                imgArr[0][1] = ((imgArr[0][1] - 116.28)/57.12)
                imgArr[0][2] = ((imgArr[0][2] - 103.53)/57.375)
                if torch.cuda.is_available():
                    output.append(model(torch.cuda.FloatTensor(imgArr)))
                else:
                    output.append(model(torch.FloatTensor(imgArr)))

            FinalOutput = imgArr
            pathStr = thisPhotoDir + '.pickle'
            with open(pathStr,'wb') as f:
                pickle.dump(FinalOutput,f)
                ndx += 1
                print("{}% complete.".format(ndx/numStacks*100))
    

Do the transformation

In [21]:
transform_to_pickle(model,topDirName)

NameError: name 'model' is not defined

Clean up the directory, removing everything but the pickle files.

In [22]:
# Assemble all the pickle files and remove the rest.
classDirs = [direc for direc in os.listdir(topDirName) if direc[0] is not '.']
for classDir in classDirs:
    newDir = os.path.join(topDirName,classDir)
    fileDirs = [direc for direc in os.listdir(newDir) if direc[0] is not '.' and not direc.lower().endswith('.pickle')]
    for fileDir in fileDirs:
        pickleFileDir = os.path.join(newDir,fileDir)
        os.system('rm -rf ' + pickleFileDir)

# Split the directory into train/test/validation sets

In [23]:
import random

In [24]:
def split_folders(inputDir, outputDir, trainValTestRatio, seed):
    # Assume inputDir is organized as 'inputDir/class/topFolder/photos'
    classes = os.listdir(inputDir)
    # Exclude folders where '.' is the first character.
    classes = [directory for directory in classes if directory[0] != '.']
    numClasses = len(classes)
    # Create output directories
    os.system('mkdir ' + outputDir)
    os.system('mkdir ' + outputDir + '/train')
    os.system('mkdir ' + outputDir + '/valid')
    os.system('mkdir ' + outputDir + '/test')
    # Pull files and put into the appropriate folders.
    for directory in classes:
        files = os.listdir(os.path.join(inputDir, directory))
        files.sort()
        random.seed(seed)
        random.shuffle(files)
        numFiles = len(files)
        # Get the indices.
        ndxAmts = np.intc((np.round(trainValTestRatio / np.sum(trainValTestRatio) * numFiles)))
        numNdxes = np.sum(ndxAmts)
        if numNdxes != numFiles:
            ndxAmts[0] += numFiles-numNdxes
        trainFiles = files[0:ndxAmts[0]]
        valFiles = files[ndxAmts[0]:ndxAmts[0]+ndxAmts[1]]
        testFiles = files[ndxAmts[0]+ndxAmts[1]:]
        # Put the training directories in the appropriate folder
        # First make the class directories.
        os.system('mkdir ' + outputDir + '/train/' + directory)
        os.system('mkdir ' + outputDir + '/valid/' + directory)
        os.system('mkdir ' + outputDir + '/test/' + directory)
        for file in trainFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/train/' + directory + '/' + file)
        for file in valFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/valid/' + directory + '/' + file)
        for file in testFiles:
            os.system('cp -r ' + inputDir + '/' + directory + '/' + file + ' ' + outputDir + '/test/' + directory + '/' + file)

In [25]:
finalDirName = 'Split_Data2_17_01_20'

In [27]:
# for idx in range(5):
#     split_folders(topDirName, finalDirName_root + str(idx),[0.8,0.1,0.1],1000 + idx)
finalDirName

'Split_Data2_17_01_20'

In [28]:
split_folders(topDirName, finalDirName,[0.8,0.1,0.1],7778)